### Single model run
Optimise parameter set and do single model run with the resulting parameters. Then illustrate results with basic plots, including the matrix scaling.

### Running over Colab
Uncomment the commands in the following cell to install in Colab.
Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment and then proceed to the following cells.

### Generating documentation
Running the final cell will take several minutes and will generate the corresponding PDF documentation.
This can be downloaded from the Files section of the sidebar.

In [ ]:
import numpy as np
import pandas as pd
import pymc as pm
from plotly import graph_objects as go
import arviz as az

from estival.model import BayesianCompartmentalModel
from estival.wrappers import pymc as epm
from estival.sampling.tools import idata_to_sampleiterator

from inputs.constants import SUPPLEMENT_PATH, PLOT_START_DATE, PRIMARY_ANALYSIS, RUNS_PATH, RUN_IDS
from aust_covid.inputs import get_ifrs
from aust_covid.model import build_model
from aust_covid.calibration import get_priors, get_targets
from aust_covid.plotting import plot_single_run_outputs, plot_example_model_matrices
from emutools.tex import DummyTexDoc, StandardTexDoc, add_image_to_doc
from emutools.utils import load_param_info, round_sigfig

In [ ]:
from summer2.parameters import Parameter, DerivedOutput, Function
from inputs.constants import INFECTION_PROCESSES, ANALYSIS_END_DATE, PLOT_START_DATE, STRAIN_STRATA
from plotly.subplots import make_subplots
from plotly.express.colors import colorbrewer

In [ ]:
from emutools.utils import get_target_from_name

In [ ]:
i_max = pd.read_hdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / 'output/results.hdf', 'likelihood')['logposterior'].idxmax()
idata = az.from_netcdf(RUNS_PATH / RUN_IDS['mob'] / 'output/calib_full_out.nc')
best_params = idata_to_sampleiterator(idata).convert('pandas').loc[i_max].to_dict()

In [ ]:
# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.9 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch main
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
max_iterations = 50

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'single_run', 'Single run', 'austcovid')
param_info = load_param_info()
param_info['value'].update(get_ifrs(DummyTexDoc()))
parameters = param_info['value'].to_dict()
intro_text = 'Results from an example run are presented in Figure \\ref{single_run_outputs}, and model construction is described in the following sections. '
app_doc.add_line(intro_text, 'Introduction')
epi_model = build_model(app_doc, param_info['abbreviations'], mobility_ext=True, cross_ref=False)
priors = get_priors(False, param_info['abbreviations'], DummyTexDoc())
prior_names = [p.name for p in priors]
targets = get_targets(DummyTexDoc())


In [ ]:
parameters.update(best_params)

In [ ]:
epi_model.run(parameters=parameters)

In [ ]:
outputs_fig = plot_single_run_outputs(epi_model, targets)
outputs_fig

In [ ]:
example_matrix_fig = plot_example_model_matrices(epi_model, parameters)
example_matrix_fig.update_layout(height=950)

In [ ]:
add_image_to_doc(outputs_fig.update_layout(showlegend=False), 'single_run_outputs', 'svg', 'Outputs from single model run', app_doc, 'Outputs')
add_image_to_doc(example_matrix_fig, 'example_matrices', 'svg', 'Dynamic mixing matrices', app_doc, 'Mixing')

In [ ]:
app_doc.write_doc()

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install inkscape texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex -shell-escape single_run
    ! biber single_run
    ! pdflatex -shell-escape single_run
    ! pdflatex -shell-escape single_run

In [ ]:
new_model = build_model(app_doc, param_info['abbreviations'], mobility_ext=True, cross_ref=False)

In [ ]:
for strain in STRAIN_STRATA:
    for process in INFECTION_PROCESSES:
        new_model.request_output_for_flow(f'{strain}X{process}', process, dest_strata={'strain': strain})
        new_model.request_function_output(f'process_{strain}X{process}', DerivedOutput(f'{strain}X{process}') / DerivedOutput('all_infection'))

In [ ]:
new_model.run(parameters=parameters)

In [ ]:
new_derived_df = new_model.get_derived_outputs_df()
cum_strain_infect = new_derived_df[[f'process_{i}X{j}' for i in STRAIN_STRATA for j in INFECTION_PROCESSES]].cumsum(axis=1)

In [ ]:
req_target = 'notifications_ma'
fig = make_subplots(2, 1, subplot_titles=['infection processes', req_target], row_heights=[0.7, 0.3], vertical_spacing=0.05, shared_xaxes=True)
for output in cum_strain_infect.columns:
    i_process = INFECTION_PROCESSES.index(output.split('X')[1])
    i_strain = STRAIN_STRATA.index(output.split('X')[0].replace('process_', ''))
    colour = f'hsl({17.0 + i_strain * 33.0}%, 70%, {30.0 + i_process * 20.0}%)'
    trace = go.Scatter(x=cum_strain_infect.index, y=cum_strain_infect[output], fill='tonexty', fillcolor=colour, line={'color': colour}, name=output)
    fig.add_traces(trace, rows=1, cols=1)
fig.update_yaxes(range=(0.0, 1.0), row=1, col=1)

fig.add_traces(new_derived_df[req_target].plot().data, rows=2, cols=1)
target = get_target_from_name(targets, req_target)
fig.add_traces(go.Scatter(x=target.index, y=target, mode='markers', name=f'{req_target} data', line={'color': 'black'}), rows=2, cols=1)
fig.update_xaxes(range=(PLOT_START_DATE, ANALYSIS_END_DATE))
fig.update_layout(height=700, margin={i: 40 for i in ['t', 'b', 'l', 'r']})